### stage_model_classifier

base: 14./stage_model_classifier_withneighbor-newfeatures

prediction models of sign classifiers on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [3]:
# bulid regression model
#classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
classifiers = ['currank','avgrank','dice','lr','lsvc','lsvcl2','rf','lrbias','xgb']
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, 
                                n_estimators = 600, colsample_bytree = 1, max_depth = 6, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [ ]:
def test_20182019():
    ### fix train
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(classifiers)
    print('cols:%s'%cols)
    retdf = pd.DataFrame([],columns=cols)


    events = set(stagedata['eventid'])

    years = ['2013','2014','2015','2016','2017','2018','2019']
    #events = ['Indy500']
    eventsname = [f'Indy500-{x}' for x in years]
    events_id={key:idx for idx, key in enumerate(eventsname)}

    #first 
    eventid = events_id['Indy500-2018']
    ignore_eventid = events_id['Indy500-2019']
    stdata_2018 = stagedata[stagedata['eventid']!=ignore_eventid]

    print('Testset = %s'%eventsname[eventid])

    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stdata_2018, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)

    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        


    eventid = events_id['Indy500-2019']
    ignore_eventid = events_id['Indy500-2018']
    stdata_2019 = stagedata[stagedata['eventid']!=ignore_eventid]

    print('Testset = %s'%eventsname[eventid])

    train2, test2, train_x2, train_y2, test_x, test_y = split_by_eventid(stdata_2019, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))

    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)

    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df]) 
    
    retdf.to_csv(f'stint_classifier_result_{suffix}.csv', float_format='%.3f')
    return retdf

def test_cv():
    cols = ['runid','trainsize','testsize','testdistribution']
    cols.extend(classifiers)
    print('cols:%s'%cols)
    retdf = pd.DataFrame([],columns=cols)


    events = set(stagedata['eventid'])

    years = ['2013','2014','2015','2016','2017','2018','2019']
    #events = ['Indy500']
    eventsname = [f'Indy500-{x}' for x in years]
    events_id={key:idx for idx, key in enumerate(eventsname)}


    for eventid in events:
        print('Testset = %s'%eventsname[eventid])

        train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
        test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
        #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
        #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

        #record
        rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

        acc = [0 for x in range(len(classifiers))]
        for idx, clf in enumerate(classifiers):
            acc[idx] = classifier_model(clf)

        rec.extend(acc)
        print('rec:%s'%rec)

        #new df
        df = pd.DataFrame([rec],columns=cols)
        retdf = pd.concat([retdf, df])        

    retdf.to_csv('crossvalid_stagedata_splitbyevent%s.csv'%suffix, float_format='%.3f')
    #df_event = retdf
    return retdf

In [4]:
#load data
_trim = 0
_include_final = True
_include_stintlen = True
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''
suffix = f'indy500-2013-2019-end{include_str}{stint_str}-t{_trim}'
output_file = f'stage-indy500-2013-2019-end{include_str}{stint_str}-t{_trim}.csv'
stagedata = pd.read_csv(output_file)

stagedata.fillna(0, inplace=True)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522 entries, 0 to 1521
Data columns (total 36 columns):
Unnamed: 0                   1522 non-null int64
target                       1522 non-null int64
stint_len                    1522 non-null int64
eventid                      1522 non-null int64
car_number                   1522 non-null int64
stageid                      1522 non-null int64
firststage                   1522 non-null int64
pit_in_caution               1522 non-null int64
start_position               1522 non-null int64
start_rank                   1522 non-null int64
start_rank_ratio             1522 non-null float64
top_pack                     1522 non-null int64
bottom_pack                  1522 non-null int64
average_rank                 1522 non-null float64
average_rank_all             1522 non-null float64
change_in_rank               1522 non-null int64
change_in_rank_all           1522 non-null float64
rate_of_change               1522 non-null int64
rat

In [5]:
stagedata.head(5)

,Unnamed: 0,target,stint_len,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,1,-12,27,0,1,1,1,0,7,15,...,22.910400,31,31,66.08150,2,8,0,14,6,3
1,2,14,32,0,1,2,1,1,7,3,...,23.822858,27,27,62.06770,-19,-21,0,-3,-21,-17
2,3,-7,32,0,1,3,1,0,7,17,...,21.857882,32,32,91.23935,11,-2,-3,7,17,19
3,4,5,30,0,1,4,1,0,7,10,...,19.394133,32,32,61.19415,-15,-9,-6,0,4,-6
4,5,-3,29,0,1,5,1,0,7,15,...,17.737505,30,30,60.85410,-3,0,0,-3,-3,0


In [6]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)


events = set(stagedata['eventid'])

years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
eventsname = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(eventsname)}


for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        
    
retdf.to_csv('crossvalid_stagedata_splitbyevent%s.csv'%suffix)
df_event = retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Indy500-2013
[*] predict with currank model
precision=[0.         0.14871795 0.        ], recall=[0. 1. 0.], f1=[0.         0.25892857 0.        ], accuracy=0.15
[*] predict with avgrank model
precision=[0.40425532 0.33333333 0.22972973], recall=[0.3877551  0.31034483 0.25      ], f1=[0.39583333 0.32142857 0.23943662], accuracy=0.33
[*] predict with dice model
precision=[0.46902655 0.2173913  0.27118644], recall=[0.54081633 0.17241379 0.23529412], f1=[0.50236967 0.19230769 0.2519685 ], accuracy=0.38
[*] predict with lr model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.61538462 0.5        0.6031746 ], recall=[0.81632653 0.03448276 0.55882353], f1=[0.70175439 0.06451613 0.58015267], accuracy=0.61
[*] predict with lsvc model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.62015504 0.5        0.609375  ], recall=[0.81632653 0.03448276 0.57352941], f1=[0.70484581 0.06451613 0.59090909], accuracy=0.62
[*] predict with lsvcl2 model


...

precision=[0.64885496 0.66666667 0.51219512], recall=[0.72649573 0.06060606 0.63636364], f1=[0.68548387 0.11111111 0.56756757], accuracy=0.60
[*] predict with lsvcl2 model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.578125   0.         0.69565217], recall=[0.94871795 0.         0.24242424], f1=[0.7184466  0.         0.35955056], accuracy=0.59
[*] predict with rf model
precision=[0.640625   0.75       0.53571429], recall=[0.7008547  0.09090909 0.68181818], f1=[0.66938776 0.16216216 0.6       ], accuracy=0.60
[*] predict with lrbias model


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


precision=[0.66412214 0.5        0.53246753], recall=[0.74358974 0.12121212 0.62121212], f1=[0.7016129  0.19512195 0.57342657], accuracy=0.61
[*] predict with xgb model
precision=[0.63909774 0.5        0.53623188], recall=[0.72649573 0.21212121 0.56060606], f1=[0.68       0.29787234 0.54814815], accuracy=0.60
rec:['Indy500-2019', 1306, 216, '+:66,0:33,-:117', 0.1527777777777778, 0.37037037037037035, 0.4861111111111111, 0.6157407407407407, 0.5972222222222222, 0.5879629629629629, 0.6018518518518519, 0.6111111111111112, 0.5972222222222222]


In [7]:
df_event

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lsvc,lsvcl2,rf,lrbias,xgb
0,Indy500-2013,1327,195,"+:68,0:29,-:98",0.148718,0.328205,0.379487,0.610256,0.615385,0.451282,0.646154,0.615385,0.641026
0,Indy500-2014,1297,225,"+:63,0:43,-:119",0.191111,0.373333,0.400000,0.591111,0.595556,0.533333,0.586667,0.595556,0.537778
0,Indy500-2015,1327,195,"+:74,0:26,-:95",0.133333,0.256410,0.446154,0.651282,0.646154,0.528205,0.661538,0.651282,0.687179
0,Indy500-2016,1268,254,"+:98,0:32,-:124",0.125984,0.362205,0.429134,0.633858,0.641732,0.543307,0.677165,0.633858,0.677165
0,Indy500-2017,1274,248,"+:72,0:49,-:127",0.197581,0.286290,0.362903,0.282258,0.306452,0.471774,0.580645,0.290323,0.616935
0,Indy500-2018,1333,189,"+:72,0:19,-:98",0.100529,0.317460,0.386243,0.656085,0.671958,0.550265,0.693122,0.656085,0.666667
0,Indy500-2019,1306,216,"+:66,0:33,-:117",0.152778,0.370370,0.486111,0.615741,0.597222,0.587963,0.601852,0.611111,0.597222


In [8]:
stagedata[(stagedata['eventid']==5) & (stagedata['car_number']==12)]

,Unnamed: 0,target,stint_len,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
1156,1327,2,18,5,12,1,1,0,3,4,...,8.527139,32,32,66.10705,-1,0,0,0,-3,-5
1157,1328,-5,44,5,12,2,1,1,3,6,...,11.242669,18,18,61.85245,-6,-5,0,6,3,3
1158,1329,1,35,5,12,3,1,0,3,1,...,23.296174,44,44,117.01525,0,0,0,-3,3,3
1159,1330,6,42,5,12,4,1,0,3,2,...,20.554273,35,35,59.24225,-3,0,0,2,1,0
1160,1331,-7,29,5,12,5,1,0,3,8,...,23.071877,42,42,59.50010,5,2,6,4,-6,5


In [9]:
### fix train
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)


events = set(stagedata['eventid'])

years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
eventsname = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(eventsname)}

#first 
eventid = events_id['Indy500-2018']
ignore_eventid = events_id['Indy500-2019']
stdata_2018 = stagedata[stagedata['eventid']!=ignore_eventid]

print('Testset = %s'%eventsname[eventid])

train, test, train_x, train_y, test_x, test_y = split_by_eventid(stdata_2018, eventid)
test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
#print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
#      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))

#record
rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

acc = [0 for x in range(len(classifiers))]
for idx, clf in enumerate(classifiers):
    acc[idx] = classifier_model(clf)

rec.extend(acc)
print('rec:%s'%rec)

#new df
df = pd.DataFrame([rec],columns=cols)
retdf = pd.concat([retdf, df])        


eventid = events_id['Indy500-2019']
ignore_eventid = events_id['Indy500-2018']
stdata_2019 = stagedata[stagedata['eventid']!=ignore_eventid]

print('Testset = %s'%eventsname[eventid])

train2, test2, train_x2, train_y2, test_x, test_y = split_by_eventid(stdata_2019, eventid)
test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))

#record
rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]

acc = [0 for x in range(len(classifiers))]
for idx, clf in enumerate(classifiers):
    acc[idx] = classifier_model(clf)

rec.extend(acc)
print('rec:%s'%rec)

#new df
df = pd.DataFrame([rec],columns=cols)
retdf = pd.concat([retdf, df]) 


cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Indy500-2018
[*] predict with currank model
precision=[0.        0.1005291 0.       ], recall=[0. 1. 0.], f1=[0.         0.18269231 0.        ], accuracy=0.10
[*] predict with avgrank model
precision=[0.38947368 0.29411765 0.23376623], recall=[0.37755102 0.26315789 0.25      ], f1=[0.38341969 0.27777778 0.24161074], accuracy=0.32
[*] predict with dice model
precision=[0.47222222 0.12       0.32142857], recall=[0.52040816 0.15789474 0.25      ], f1=[0.49514563 0.13636364 0.28125   ], accuracy=0.38
[*] predict with lr model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.67669173 0.25       0.79166667], recall=[0.91836735 0.10526316 0.52777778], f1=[0.77922078 0.14814815 0.63333333], accuracy=0.69
[*] predict with lsvc model


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.625      0.         0.83783784], recall=[0.96938776 0.         0.43055556], f1=[0.76       0.         0.56880734], accuracy=0.67
[*] predict with lsvcl2 model


...

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision=[0.68421053 0.         0.5       ], recall=[0.66666667 0.         0.77272727], f1=[0.67532468 0.         0.60714286], accuracy=0.60
[*] predict with rf model
precision=[0.62773723 0.66666667 0.53424658], recall=[0.73504274 0.12121212 0.59090909], f1=[0.67716535 0.20512821 0.56115108], accuracy=0.60
[*] predict with lrbias model


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished


precision=[0.67391304 0.44444444 0.57971014], recall=[0.79487179 0.12121212 0.60606061], f1=[0.72941176 0.19047619 0.59259259], accuracy=0.63
[*] predict with xgb model
precision=[0.66666667 0.5        0.57142857], recall=[0.76923077 0.27272727 0.54545455], f1=[0.71428571 0.35294118 0.55813953], accuracy=0.62
rec:['Indy500-2019', 1117, 216, '+:66,0:33,-:117', 0.1527777777777778, 0.37037037037037035, 0.48148148148148145, 0.6342592592592593, 0.6388888888888888, 0.5972222222222222, 0.5972222222222222, 0.6342592592592593, 0.625]


,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lsvc,lsvcl2,rf,lrbias,xgb
0,Indy500-2018,1117,189,"+:72,0:19,-:98",0.100529,0.31746,0.380952,0.687831,0.666667,0.666667,0.687831,0.671958,0.650794
0,Indy500-2019,1117,216,"+:66,0:33,-:117",0.152778,0.37037,0.481481,0.634259,0.638889,0.597222,0.597222,0.634259,0.625000


In [10]:
retdf.to_csv(f'stint_classifier_result_{suffix}.csv', float_format='%.3f')

In [11]:
retdf

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lsvc,lsvcl2,rf,lrbias,xgb
0,Indy500-2018,1117,189,"+:72,0:19,-:98",0.100529,0.31746,0.380952,0.687831,0.666667,0.666667,0.687831,0.671958,0.650794
0,Indy500-2019,1117,216,"+:66,0:33,-:117",0.152778,0.37037,0.481481,0.634259,0.638889,0.597222,0.597222,0.634259,0.625000
